## Recency, Frequency, Monetary Value analysis

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [4]:
online = pd.read_csv('Data/online.csv', index_col=0, parse_dates=['InvoiceDate'])
online.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
416792,572558,22745,POPPY'S PLAYHOUSE BEDROOM,6,2011-10-25 08:26:00,2.10,14286,United Kingdom
482904,577485,23196,VINTAGE LEAF MAGNETIC NOTEPAD,1,2011-11-20 11:56:00,1.45,16360,United Kingdom
263743,560034,23299,FOOD COVER WITH BEADS SET 2,6,2011-07-14 13:35:00,3.75,13933,United Kingdom
495549,578307,72349B,SET/6 PURPLE BUTTERFLY T-LIGHTS,1,2011-11-23 15:53:00,2.10,17290,United Kingdom
204384,554656,21756,BATH BUILDING BLOCK WORD,3,2011-05-25 13:36:00,5.95,17663,United Kingdom


In [5]:
online.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70864 entries, 416792 to 312243
Data columns (total 8 columns):
InvoiceNo      70864 non-null int64
StockCode      70864 non-null object
Description    70864 non-null object
Quantity       70864 non-null int64
InvoiceDate    70864 non-null datetime64[ns]
UnitPrice      70864 non-null float64
CustomerID     70864 non-null int64
Country        70864 non-null object
dtypes: datetime64[ns](1), float64(1), int64(3), object(3)
memory usage: 4.9+ MB


In [6]:
online['TotalSum'] = online['Quantity'] * online['UnitPrice']

In [7]:
online.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalSum
416792,572558,22745,POPPY'S PLAYHOUSE BEDROOM,6,2011-10-25 08:26:00,2.10,14286,United Kingdom,12.60
482904,577485,23196,VINTAGE LEAF MAGNETIC NOTEPAD,1,2011-11-20 11:56:00,1.45,16360,United Kingdom,1.45
263743,560034,23299,FOOD COVER WITH BEADS SET 2,6,2011-07-14 13:35:00,3.75,13933,United Kingdom,22.50
495549,578307,72349B,SET/6 PURPLE BUTTERFLY T-LIGHTS,1,2011-11-23 15:53:00,2.10,17290,United Kingdom,2.10
204384,554656,21756,BATH BUILDING BLOCK WORD,3,2011-05-25 13:36:00,5.95,17663,United Kingdom,17.85


### Data preparation steps

We're starting with a pre-processed online DataFrame with only the latest 12
months of data:

In [9]:
online = online.loc[(online['InvoiceDate'] >= '2010-12-10') & (online['InvoiceDate'] < '2011-12-13')]

In [10]:
print('Min:{}; Max:{}'.format(min(online.InvoiceDate),
                              max(online.InvoiceDate)))

Min:2010-12-10 09:33:00; Max:2011-12-09 12:49:00


In [12]:
# Let's create a hypothetical snapshot_day data as if we're doing analysis recently
snapshot_date = max(online.InvoiceDate) + datetime.timedelta(days=1)

snapshot_date

Timestamp('2011-12-10 12:49:00')

### Calculate RFM metrics

In [15]:
# Aggregate data on a customer level
datamart = online.groupby(['CustomerID']).agg({
                            'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
                            'InvoiceNo': 'count',
                            'TotalSum': 'sum'})

# Rename columns for easier interpretation
datamart.rename(columns = {'InvoiceDate': 'Recency',
                        'InvoiceNo': 'Frequency',
                        'TotalSum': 'MonetaryValue'}, inplace=True)

# Check the first rows
datamart.head()

,Recency,Frequency,MonetaryValue
CustomerID,,,
12747,2,25,948.70
12748,1,888,7046.16
12749,4,37,813.45
12820,3,17,268.02
12822,71,9,146.15


### Building RFM segments

Will calculate quartile value for each column and name then R, F, M

##### Recency quartile

In [17]:
r_labels = range(4, 0, -1)

r_quartiles = pd.qcut(datamart['Recency'], 4, labels = r_labels)

datamart = datamart.assign(R = r_quartiles.values)

datamart.head()

,Recency,Frequency,MonetaryValue,R
CustomerID,,,,
12747,2,25,948.70,4
12748,1,888,7046.16,4
12749,4,37,813.45,4
12820,3,17,268.02,4
12822,71,9,146.15,2


##### Frequency and Monetary quartiles

In [20]:
f_labels = range(1,5)
m_labels = range(1,5)

f_quartiles = pd.qcut(datamart['Frequency'], 4, labels = f_labels)
m_quartiles = pd.qcut(datamart['MonetaryValue'], 4, labels = m_labels)

datamart = datamart.assign(F = f_quartiles.values)
datamart = datamart.assign(M = m_quartiles.values)

datamart.head()

,Recency,Frequency,MonetaryValue,R,F,M
CustomerID,,,,,,
12747,2,25,948.70,4,4,4
12748,1,888,7046.16,4,4,4
12749,4,37,813.45,4,4,4
12820,3,17,268.02,4,3,3
12822,71,9,146.15,2,2,3


#### Build RFM Segment and RFM Score

In [29]:
def join_rfm(x): return str(x['R']) + str(x['F']) + str(x['M'])

datamart['RFM_Segment'] = datamart.apply(join_rfm, axis=1)

datamart['RFM_Score'] = datamart[['R','F','M']].sum(axis=1)

datamart.head(10)

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Segment,RFM_Score
CustomerID,,,,,,,,
12747,2,25,948.70,4,4,4,444,12.0
12748,1,888,7046.16,4,4,4,444,12.0
12749,4,37,813.45,4,4,4,444,12.0
12820,3,17,268.02,4,3,3,433,10.0
12822,71,9,146.15,2,2,3,223,7.0
12823,297,1,306.00,1,1,3,113,5.0
12824,60,6,106.38,2,2,2,222,6.0
12826,3,16,272.66,4,3,3,433,10.0
12827,33,2,44.55,3,1,1,311,5.0


In [28]:
datamart.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3643 entries, 12747 to 18287
Data columns (total 8 columns):
Recency          3643 non-null int64
Frequency        3643 non-null int64
MonetaryValue    3643 non-null float64
R                3643 non-null category
F                3643 non-null category
M                3643 non-null category
RFM_Segment      3643 non-null object
RFM_Score        3643 non-null float64
dtypes: category(3), float64(2), int64(2), object(1)
memory usage: 181.8+ KB


### Analyzing RFM segments

In [34]:
datamart.groupby('RFM_Segment').size().sort_values(ascending=False)[:10]

RFM_Segment
444    367
111    345
211    167
344    162
233    128
222    125
333    122
311    119
122    117
433    112
dtype: int64

In [38]:
datamart.RFM_Segment.value_counts().head(10)

444    367
111    345
211    167
344    162
233    128
222    125
333    122
311    119
122    117
433    112
Name: RFM_Segment, dtype: int64

### Summary metrics per RFM Score

In [40]:
datamart.groupby('RFM_Score').agg({
                'Recency': 'mean',
                'Frequency': 'mean',
                'MonetaryValue': ['mean', 'count'] }).round(1)

Recency Frequency MonetaryValue      
             mean      mean          mean count
RFM_Score                                      
3.0         246.4       2.1          28.4   345
4.0         162.3       3.1          47.9   335
5.0         137.4       4.3          77.5   396
6.0         101.3       6.3         147.1   439
7.0          76.5       8.4         158.7   391
8.0          62.2      12.8         197.7   373
9.0          46.5      16.8         330.8   344
10.0         31.2      24.1         449.2   358
11.0         21.5      39.3         704.8   295
12.0          7.3      76.0        1664.9   367

### Grouping into named segments

- RFM_Score >= 9 ===> Gold
- RFM_Score >= 5 and RFM_Score < 9 ===> Silver
- RFM_Score < 5 ===> Bronze

In [50]:
# Note the bins!!!
datamart['General_Segment'] = pd.cut(datamart['RFM_Score'], bins=[2, 4, 8, 13], labels=['Bronze', 'Silver', 'Gold'])

In [52]:
datamart.head()

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Segment,RFM_Score,General_Segment
CustomerID,,,,,,,,,
12747,2,25,948.70,4,4,4,444,12.0,Gold
12748,1,888,7046.16,4,4,4,444,12.0,Gold
12749,4,37,813.45,4,4,4,444,12.0,Gold
12820,3,17,268.02,4,3,3,433,10.0,Gold
12822,71,9,146.15,2,2,3,223,7.0,Silver


In [53]:
datamart.groupby('General_Segment').agg({
                'Recency': 'mean',
                'Frequency': 'mean',
                'MonetaryValue': ['mean', 'count']
                }).round(1)

Recency Frequency MonetaryValue      
                   mean      mean          mean count
General_Segment                                      
Bronze            205.0       2.6          38.0   680
Silver             95.1       7.9         144.5  1599
Gold               26.5      39.5         801.7  1364

Source: Datacamp